## Test for Aginic

### Steps
1. Generator data for Freshdesk
2. Store data into SQLite
3. SQL Script to generate attributes (Specifically says SQL so no Python assumed).
    - Time spent Open - So the current time minus the ticket open time; if it has been closed, final ticket time minus original time (Excluding this for speed). 
    - Time spent Waiting on Customer - Ticket of waiting on customers current time minus ticket open time.
    - Time spent waiting for response (Pending Status) - 
    - Time till resolution - time to solved.
    - Time to first response (First time to first ticket time).
4. Programs must be able to be run in a bash script or equivilent.

In [1]:
!pip install faker

### Libraries Required

In [11]:
from datetime import datetime, timedelta
from faker import Faker
import pandas as pd
from random import randrange
import json
import sqlite3



### Ticket Generator Function

In [167]:
def ticket_gen(n):
        
        """ This creates a random set of tickets."""  
        
        #This is the different words we'll be using for various generated data sets
        faker = Faker()
        df = []
        meta = {}
        activ = {}
        key = []
        users = ['user', 'admin', 'customer']
        activities = ['note', 'admin', 'customer']
        category = ['Phone', 'computer', 'jackets']
        product = ['mobile', 'tigimon', 'tamigotcha']
        status = ['open', 'closed', 'resolved','waiting for customer', 'waiting for third party', "pending"]
        descr_act = []
        activity = []
        meta_data = []
        for n in range(n):
        
        
            #key.append(n)
            #this creates the intial metadata
            
            #Time is set from the time_origin, which is when the first ticket is made,
            #and then added onto at random intervals 
            
            time_origin = faker.date_time_this_century()
    
            activities = randrange(10)
            work_type  = randrange(1,4)
           
            
            desc_act_mwap ={'key':n,
                         'performed_at': faker.date_time_this_century(),
                         'ticket_id': randrange(80,1000),
                         'performer_type': faker.words(1, users, True)[0],
                         'performer_id': randrange(80,10000) }
            descr_act.append(desc_act_mwap)


            activity_next = time_origin
            for x in range(activities):
                #depending on the total activities, this creates what information is in those activities
                activity_next = activity_next + timedelta(days=randrange(10))

                if work_type == 1:
                    #if the ticket type is note, as per example

                    act_data_note =  {'key':n,'performed_at': activity_next,
                             'id': randrange(80,1000),
                             'category': faker.words(1, category, True)[0],
                             'type': randrange(20)}
                    activity.append(act_data_note)

                    x+=1
                    activity_next = activity_next + timedelta(days=randrange(3))

                else:
                                        #if the ticket type is shipping, as per example

                    act_data_shipping =  {
                        'key':n,
                        'performed_at': time_origin,
                        'shipping_address': faker.address(),
                        'shipping_date': activity_next,
                        'category': faker.words(1, users, True)[0],
                        'priority': randrange(5),
                        'status': faker.words(1, status, True)[0],
                        'contacted_customer': faker.pybool(),
                        'source': randrange(5),
                        'agent_id': randrange(2000),
                        'requester': randrange(2000),
                        'product': faker.words(1, product, True)[0]}
                    activity.append(act_data_shipping)
                    activity_next = activity_next + timedelta(days=randrange(3))



                    x+=1
                    
            meta_data_make = {'key':n,'start_at': time_origin, 'end_at': activity_next,
                                   'activities_count': randrange(100) }
            meta_data.append(meta_data_make)
            
            time_origin = activity_next + timedelta(days=randrange(10))

                    
            #when this loop has finished, we append the generated data to the dataframe.
           # df.append({'metadata': {'key':{n},'start_at': {time_origin}, 'end_at': {activity_next},
                  #                 'activities_count': {randrange(100)} },
              #        'activities_data':[descr_act,activity]})
        act_df = pd.DataFrame(activity)#.set_index(['key'])
        desc_df = pd.DataFrame(descr_act)#.set_index(['key'])
        meta_df = pd.DataFrame(meta_data)#.set_index(['key'])
        return act_df,desc_df,meta_df


##### Example of Output.

In [168]:
act_df,desc_df,meta_df  = ticket_gen(1000)
act_df

,key,performed_at,shipping_address,shipping_date,category,priority,status,contacted_customer,source,agent_id,requester,product,id,type
0,1,2014-10-16 12:33:02,913 Brianna Crescent Apt. 768\nWest Laurieside...,2014-10-19 12:33:02,customer,1.0,pending,True,4.0,1108.0,1929.0,tamigotcha,NaN,NaN
1,2,2006-04-08 08:02:52,"70345 Steven Lodge Suite 162\nLawrenceville, P...",2006-04-09 08:02:52,user,1.0,resolved,False,1.0,440.0,452.0,tamigotcha,NaN,NaN
2,2,2006-04-08 08:02:52,"340 Alicia Ports Apt. 785\nWest Sarah, OH 90046",2006-04-18 08:02:52,admin,4.0,open,False,3.0,21.0,1093.0,tigimon,NaN,NaN
3,2,2006-04-08 08:02:52,"6423 Yolanda Viaduct\nWest Daniel, AR 52847",2006-04-24 08:02:52,admin,2.0,waiting for third party,True,0.0,880.0,802.0,tamigotcha,NaN,NaN
4,2,2006-04-08 08:02:52,"PSC 4304, Box 2709\nAPO AA 35610",2006-05-04 08:02:52,user,3.0,waiting for third party,False,4.0,545.0,1848.0,mobile,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4591,999,2014-08-22 17:18:56,"309 Hawkins Rue\nJenniferport, OR 01254",2014-08-27 17:18:56,user,1.0,resolved,True,4.0,1171.0,341.0,tamigotcha,NaN,NaN
4592,999,2014-08-22 17:18:56,"35364 Andrew Harbors\nPort Suzanne, SC 31092",2014-09-06 17:18:56,user,1.0,waiting for customer,False,0.0,917.0,1710.0,mobile,NaN,NaN
4593,999,2014-08-22 17:18:56,"41300 Douglas Dale\nNew Jesusbury, NC 78787",2014-09-09 17:18:56,admin,1.0,pending,True,3.0,521.0,801.0,tigimon,NaN,NaN
4594,999,2014-08-22 17:18:56,"6342 Boyd Square Apt. 680\nAshleystad, TX 01687",2014-09-12 17:18:56,user,2.0,waiting for third party,False,3.0,222.0,1630.0,tigimon,NaN,NaN


In [169]:
desc_df

,key,performed_at,ticket_id,performer_type,performer_id
0,0,2000-02-09 04:20:12,964,customer,4038
1,1,2013-06-27 21:21:58,118,admin,9413
2,2,2011-04-10 04:30:38,780,customer,5898
3,3,2016-08-07 00:37:23,140,user,7281
4,4,2003-01-22 06:26:30,695,admin,4490
...,...,...,...,...,...
995,995,2014-01-04 06:53:27,800,user,7495
996,996,2008-03-23 22:56:09,772,user,5058
997,997,2016-10-29 10:33:18,769,customer,7307
998,998,2011-02-21 03:49:19,742,user,3200


### Save Generated Data to Json

In [170]:
#timestr = time.strftime("%Y%m%d-%H%M%S") This would be how I would make the files historical.

#combined_df.to_json('generated_data.json', orient='records', lines=True)
meta_df.to_json('metadata.json', orient='records', lines=True)
act_df.to_json('activities_data.json', orient='records', lines=True)
desc_df.to_json('description_data.json', orient='records', lines=True)

In [158]:
meta_df

,key,start_at,end_at,activities_count
0,0,2013-04-24 02:41:32,2013-05-08 02:41:32,54
1,1,2004-05-22 05:15:46,2004-07-27 05:15:46,36
2,2,2011-01-13 11:59:31,2011-03-06 11:59:31,24
3,3,2007-10-26 22:20:34,2007-12-07 22:20:34,29
4,4,2018-07-24 12:33:09,2018-09-04 12:33:09,16
...,...,...,...,...
995,995,2013-11-27 05:28:33,2013-12-21 05:28:33,28
996,996,2009-01-28 10:07:51,2009-02-27 10:07:51,3
997,997,2004-02-25 04:44:17,2004-02-25 04:44:17,5
998,998,2002-07-27 00:05:34,2002-07-27 00:05:34,39


### Take JSON and move it to SQLite

1. Read Json
2. Take Json and convert into format easily insertable to SQLite.
3. Create SQL Queries and Insert data to Database.

In [171]:
db=sqlite3.connect('FreshDeskTest.sqlite3')
tablename = ["metadata","activities_data","description_data"]
table_length = len(tablename)

n = 0
for n in range(table_length):
    
    json_data = []
    for line in open(tablename[n]+'.json', 'r', encoding='utf-8-sig'):
        json_data.append(json.loads(line))



    #Aim of this block is to get the list of the columns in the JSON file.
    columns = []
    column = []
    print(columns)
    for data in json_data:
        column = list(data.keys())
        for col in column:
            if col not in columns:
                columns.append(col)

    #Here we get values of the columns in the JSON file in the right order.   
    value = []
    values = [] 
    for data in json_data:
        for i in columns:
            value.append(str(dict(data).get(i)))   
        values.append(list(value)) 
        value.clear()

    if n == 0:

       #Time to generate the create and insert queries and apply it to the sqlite3 database       
        create_query = "create table if not exists metadata ({0})".format(" text,".join(columns))
        insert_query = "insert into metadata ({0}) values (?{1})".format(",".join(columns), ",?" * (len(columns)-1))    
        print("insert has started at " + str(datetime.now()))  
        c = db.cursor()   
        c.execute(create_query)
        c.executemany(insert_query , values)
        values.clear()
        db.commit()
        c.close()
        print("insert has completed at " + str(datetime.now())) 
    elif n ==1 :
        #Time to generate the create and insert queries and apply it to the sqlite3 database       
        create_query = "create table if not exists activities_data ({0})".format(" text,".join(columns))
        insert_query = "insert into activities_data ({0}) values (?{1})".format(",".join(columns), ",?" * (len(columns)-1))    
        print("insert has started at " + str(datetime.now()))  
        c = db.cursor()   
        c.execute(create_query)
        c.executemany(insert_query , values)
        values.clear()
        db.commit()
        c.close()
        print("insert has completed at " + str(datetime.now())) 
    else:
        #Time to generate the create and insert queries and apply it to the sqlite3 database       
        create_query = "create table if not exists description_data ({0})".format(" text,".join(columns))
        insert_query = "insert into description_data ({0}) values (?{1})".format(",".join(columns), ",?" * (len(columns)-1))    
        print("insert has started at " + str(datetime.now()))  
        c = db.cursor()   
        c.execute(create_query)
        c.executemany(insert_query , values)
        values.clear()
        db.commit()
        c.close()
        print("insert has completed at " + str(datetime.now())) 
    
    n+=1


[]
insert has started at 2020-06-09 18:35:01.697493
insert has completed at 2020-06-09 18:35:01.716017
[]
insert has started at 2020-06-09 18:35:01.849512
insert has completed at 2020-06-09 18:35:01.874919
[]
insert has started at 2020-06-09 18:35:01.889621
insert has completed at 2020-06-09 18:35:01.903437


##### Open Data now inserted to ensure it is working properly.

In [172]:
# Create your connection.
cnx = sqlite3.connect('FreshDeskTest.sqlite3')
df = pd.read_sql_query("SELECT * FROM activities_data", cnx)


In [173]:
df.set_index(['key'])

,performed_at,shipping_address,shipping_date,category,priority,status,contacted_customer,source,agent_id,requester,product,id,type
key,,,,,,,,,,,,,
0,1366771292000,"577 Ana Plaza Apt. 791\nEast Debbie, NY 81034",1367289692000,user,3.0,waiting for customer,True,4.0,378.0,836.0,tamigotcha,None,None
0,1366771292000,"06931 Samuel Motorway\nCarrietown, IN 89445",1367894492000,admin,0.0,waiting for third party,True,4.0,1686.0,689.0,mobile,None,None
1,1085980546000,None,None,computer,None,None,None,None,None,None,None,87.0,16.0
1,1086844546000,None,None,Phone,None,None,None,None,None,None,None,821.0,9.0
1,1086844546000,None,None,jackets,None,None,None,None,None,None,None,617.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999,1408727936000,"309 Hawkins Rue\nJenniferport, OR 01254",1409159936000,user,1.0,resolved,True,4.0,1171.0,341.0,tamigotcha,None,None
999,1408727936000,"35364 Andrew Harbors\nPort Suzanne, SC 31092",1410023936000,user,1.0,waiting for customer,False,0.0,917.0,1710.0,mobile,None,None
999,1408727936000,"41300 Douglas Dale\nNew Jesusbury, NC 78787",1410283136000,admin,1.0,pending,True,3.0,521.0,801.0,tigimon,None,None
